In [ ]:
!pip install hub

In [ ]:
!pip install av

In [124]:
import os
import hub
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import librosa
import librosa.display
import soundfile as sf
import pywt
import pickle

import random
from random import seed, random, randint, sample

from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [132]:
import tensorflow.keras as keras
from keras import backend as K
from keras.models import Model, load_model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, GlobalMaxPool1D, Activation, MaxPool1D, Conv1D, Flatten, BatchNormalization
from keras.regularizers import l2
#from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
ds = hub.load("hub://activeloop/spoken_mnist")

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/spoken_mnist



\

hub://activeloop/spoken_mnist loaded successfully.



In [49]:
audio = []
yy = []
for i in tqdm(range(3000)) :
  if ds[i]['speakers'].data()['value'] == 'george':
    audio.append(ds[i]['audio'].numpy()[:,0])
    yy.append(0)
  if ds[i]['speakers'].data()['value'] == 'jackson':
    audio.append(ds[i]['audio'].numpy()[:,0])
    yy.append(1)
  if ds[i]['speakers'].data()['value'] == 'lucas':
    audio.append(ds[i]['audio'].numpy()[:,0])
    yy.append(2)

100%|██████████| 3000/3000 [00:18<00:00, 164.69it/s]


In [50]:
audio_train, audio_test, y_train, y_test = train_test_split(audio, yy, test_size=0.3)

In [51]:
print("Training Data class distribution: ", np.unique(y_train, return_counts=True))
print("Testing Data class distribution: ", np.unique(y_test, return_counts=True))

Training Data class distribution:  (array([0, 1, 2]), array([342, 348, 360]))
Testing Data class distribution:  (array([0, 1, 2]), array([158, 152, 140]))


In [46]:
def compute_wavelet_features(X) :

    # Define a few parameters
    wavelet = 'morl' # wavelet type: morlet
    sr = 8000 # sampling frequency: 8KHz
    widths = np.arange(1, 256) # scales for morlet wavelet
    dt = 1/sr # timestep difference

    frequencies = pywt.scale2frequency(wavelet, widths) / dt # Get frequencies corresponding to scales

    # Create a filter to select frequencies between 80Hz and 5KHz
    upper = ([x for x in range(len(widths)) if frequencies[x] > 1000])[-1]
    lower = ([x for x in range(len(widths)) if frequencies[x] < 80])[0]
    widths = widths[upper:lower] # Select scales in this frequency range

    # Compute continuous wavelet transform of the audio numpy array
    wavelet_coeffs, freqs = pywt.cwt(X, widths, wavelet = wavelet, sampling_period=dt)
    # print(wavelet_coeffs.shape)
    # sys.exit(1)

    # Split the coefficients into frames of length 800
    start = 0
    end = wavelet_coeffs.shape[1]
    frames = []
    frame_size = 400
    count = 0

    while start+frame_size <= end-1 :

        f = wavelet_coeffs[:,start:start+frame_size]

        # Total samples in a frame will not be a multiple of 800 everytime. If the last frame length is less than 800, we can skip it.
        assert f.shape[1] == frame_size # assert frame lengths are equal to the frame_size parameter

        frames.append(f)
        start += frame_size


    # Convert frames to numpy array
    frames = np.array(frames)
    frames = frames.reshape((len(frames), wavelet_coeffs.shape[0], frame_size))

    return frames

In [120]:
### Compute Training data features. We have each sample into frames of length 400

indices = []
WaveletFeatTrain = [] # Store wavelet features
WaveletYTrain = [] # Store class labels corresponding to wavelet features from an audio sample
uniq_id = []
count = 0

for i in range(3) :

    array = np.array(y_train)
    ind = np.where(array == i)
    seed(i)
    ind = ind[0].tolist()
    ind = sample(ind, 100)
    audio_samples = []

    for l in range(len(ind)):
      audio_samples.append(audio_train[l])

    num_rand_samp = 100

    for j in tqdm(range(len(audio_samples))) :

        # print("i ", i, " j ", j, "/", len(audio_samples))
        curr_sample = audio_samples[j]
        #seq, _ = librosa.load(curr_sample)
        F = compute_wavelet_features(curr_sample)
        F = F.astype(np.float16)

        # Generate target labels corresponding to the frames of each sample
        indices = np.arange(0, len(F), 1)
        indices = indices.tolist()
        indices = sample(indices, min(num_rand_samp, len(indices)))
        F = F[indices]
        uniq_id += [count] * len(F)
        WaveletYTrain += [i] * len(F)

        if count == 0 :
            WaveletFeatTrain = F
        else :
            WaveletFeatTrain = np.concatenate((WaveletFeatTrain, F), axis=0)

        count += 1



print("X: ", WaveletFeatTrain.shape)

WaveletYTrain = np.array(WaveletYTrain) # Convert to numpy array
uniq_id = np.array(uniq_id)
print("Y: ", WaveletYTrain.shape, " unique: ", np.unique(WaveletYTrain, return_counts=True))
# Write all features to a .npz file
np.savez_compressed(os.getcwd()+"/training_features", a=WaveletFeatTrain, b=WaveletYTrain, c=uniq_id)


 10%|█         | 10/100 [00:00<00:05, 15.42it/s]<ipython-input-120-2306bf3fabde>:29: RuntimeWarning: overflow encountered in cast
  F = F.astype(np.float16)
100%|██████████| 100/100 [00:17<00:00,  5.76it/s]


X:  (2919, 76, 400)
Y:  (2919,)  unique:  (array([0, 1, 2]), array([973, 973, 973]))


In [121]:
### Compute Testing data features

WaveletFeatTest = [] # Store wavelet features. We have each sample into frames of length 400
WaveletYTest = [] # Store class labels corresponding to wavelet features from an audio sample
uniq_id = []

for i in tqdm(range(len(audio_test))) :

    curr_sample = audio_test[i]
    #seq, _ = librosa.load(curr_sample)
    curr_target = y_test[i]
    F = compute_wavelet_features(curr_sample)

    # Generate target labels corresponding to the frames of each sample
    WaveletYTest += [curr_target] * len(F)
    uniq_id += [i] * len(F)

    if i == 0 :
        WaveletFeatTest = F
    else :
        WaveletFeatTest = np.concatenate((WaveletFeatTest, F), axis=0)

WaveletYTest = np.array(WaveletYTest) # Convert to numpy array
uniq_id = np.array(uniq_id)
print("X: ", WaveletFeatTest.shape, "  y: ", WaveletYTest.shape)

WaveletFeatTest = WaveletFeatTest.astype(np.float16)

# Write all features to a .npz file
np.savez_compressed(os.getcwd()+"/testing_features", a=WaveletFeatTest, b=WaveletYTest, c=uniq_id)

100%|██████████| 450/450 [03:12<00:00,  2.34it/s]


X:  (4268, 76, 400)   y:  (4268,)


<ipython-input-121-c0380dde07c3>:27: RuntimeWarning: overflow encountered in cast
  WaveletFeatTest = WaveletFeatTest.astype(np.float16)


### Crear modelo

In [122]:
def create_model(row, col) :

    n_filters = 32
    filter_width = 3
    dilation_rates = [2**i for i in range(6)] * 2

    # define an input history series and pass it through a stack of dilated causal convolution blocks
    history_seq = Input(shape=(row, col))
    x = history_seq

    skips = []
    count = 0
    # x = GaussianNoise(0.01)(x)
    for dilation_rate in dilation_rates:

        # preprocessing - equivalent to time-distributed dense

        # filter
        x = Conv1D(filters=n_filters,
                    kernel_size=filter_width,
                    padding='causal',
                    dilation_rate=dilation_rate, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))(x)

        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    out = Conv1D(16, 3, padding='same', kernel_initializer= 'random_normal', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))(x)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = GlobalMaxPool1D()(out)

    out = Dense(3, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))(out)
    out = Activation('softmax')(out)

    model = Model(history_seq, out)

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

model = create_model(400, 76)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400, 76)]         0         
                                                                 
 conv1d (Conv1D)             (None, 400, 32)           7328      
                                                                 
 batch_normalization (Batch  (None, 400, 32)           128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 400, 32)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 400, 32)           3104      
                                                                 
 batch_normalization_1 (Bat  (None, 400, 32)           128       
 chNormalization)                                            

### train model

In [125]:
# Load the data
training_data = np.load(os.getcwd()+"/training_features.npz")
X = training_data['a']
y = training_data['b']

X = X.transpose(0,2,1) # Put data in correct format: Num_samples x timesteps x features
y = to_categorical(y) # Convert class labels to categorial vectors
print("X ", X.shape, "y ", y.shape)


X  (2919, 400, 76) y  (2919, 3)


In [126]:
# Standardize the data
mean = X.mean()
std = X.std()
X = (X-mean)/ std

print("Mean ", mean, " STD ", std, X.mean(), X.std())

X = X.astype(np.float16)

y = y.astype(np.uint8)

print("Input shapes ", X.shape, y.shape)


/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:118: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:152: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:152: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


Mean  nan  STD  nan nan nan
Input shapes  (2919, 400, 76) (2919, 3)


In [128]:
# Write the standard deviation and mean in a pickle file
f = open(os.getcwd()+'/speaker_mean_std.pkl', 'wb')
pickle.dump([mean, std, y], f)
f.close()

In [129]:
### ANTES DEL MODELO

r,c = X[0].shape

# Split data into training and validation
X1, Xval, y1, yval = train_test_split(X, y, test_size=0.20)#, random_state=int(time.time()))

# Use 5-fold cross validation
kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [131]:
count = 0

# Train the model
for train, test in kfold.split(X1, np.argmax(y1, axis= -1)):

    print("K Fold Step  ", count)
    model.fit(X1[train], y1[train], validation_data= (X1[test], y1[test]), batch_size= 128, epochs= 80, verbose= 2)
    model.save(os.getcwd()+"/speaker_classifier.h5")

    count += 1

    scores = model.evaluate(Xval, yval, verbose=0)
    print("Metrics : ", scores)

K Fold Step   0
Epoch 1/80
15/15 - 36s - loss: nan - val_loss: nan - 36s/epoch - 2s/step
Epoch 2/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 3/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 4/80
15/15 - 21s - loss: nan - val_loss: nan - 21s/epoch - 1s/step
Epoch 5/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 6/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 7/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 8/80
15/15 - 21s - loss: nan - val_loss: nan - 21s/epoch - 1s/step
Epoch 9/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 10/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 11/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 12/80
15/15 - 20s - loss: nan - val_loss: nan - 20s/epoch - 1s/step
Epoch 13/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 14/80
15/15 - 18s - loss:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Metrics :  nan
K Fold Step   1
Epoch 1/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 2/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 3/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 4/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 5/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 6/80
15/15 - 20s - loss: nan - val_loss: nan - 20s/epoch - 1s/step
Epoch 7/80
15/15 - 17s - loss: nan - val_loss: nan - 17s/epoch - 1s/step
Epoch 8/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 9/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 10/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 11/80
15/15 - 19s - loss: nan - val_loss: nan - 19s/epoch - 1s/step
Epoch 12/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 13/80
15/15 - 18s - loss: nan - val_loss: nan - 18s/epoch - 1s/step
Epoch 14/80
15/1

In [133]:
model = load_model(os.getcwd()+"/speaker_classifier.h5", compile= False)

print(model.summary())

# Load the standard deviation and mean
f = open(os.getcwd()+'/speaker_mean_std.pkl', 'rb')
mean, std, poss_knnn = pickle.load(f)
f.close()

testing_data = np.load(os.getcwd()+"/testing_features.npz")
X = testing_data['a']
y = testing_data['b']
ind = testing_data['c']
unq_ind = np.unique(ind)

X = X.astype(np.float16)

X = X.transpose(0,2,1) # Put data in correct format: Num_samples x timesteps x features
X = (X-mean)/ std

# Predict
ypred = model.predict(X)
ypred = np.argmax(ypred, axis=-1)
ypred = ypred.flatten()
new_pred = []
new_truth = []

# Find unique ids and assign class based on majority vote from all the frames
for i in range(len(unq_ind)) :
    curr = unq_ind[i]
    indices, = np.where(ind == curr)
    t = y[indices]
    t = t[0]

    p = ypred[indices]
    # p1 = ypred1[indices]
    # p2 = ypred2[indices]
    # p3 = ypred3[indices]
    # p4 = ypred4[indices]

    # all_model_pred = [get_best_candidate(x, t) for x in [p1, p2, p3, p4]]
    # un, fr = np.unique(all_model_pred, return_counts=True)

    un, fr = np.unique(p, return_counts=True)
    new_pred.append(un[np.argmax(fr)])
    print("Truth ", t, " Pred ", un, fr)

    new_truth.append(t)

new_truth = np.array(new_truth)
new_pred = np.array(new_pred)

# Print classification report
rep = classification_report(new_truth, new_pred, target_names=['speaker1', 'speaker2', 'speaker3'])
print(rep)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400, 76)]         0         
                                                                 
 conv1d (Conv1D)             (None, 400, 32)           7328      
                                                                 
 batch_normalization (Batch  (None, 400, 32)           128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 400, 32)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 400, 32)           3104      
                                                                 
 batch_normalization_1 (Bat  (None, 400, 32)           128       
 chNormalization)                                            

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
